# CNLS-DDF (without undesirable outputs)

Until now, the CNLS/StoNED framework has been presented in the single output, 
multiple input setting. In this part we describe the CNLS estimator 
within the directional distance function (DDF) framework, Chambers et al. (1996,1998).

Consider the following QP problem

   \begin{align*}
        & \underset{\alpha, \beta, \varepsilon} {min} \sum_{i=1}^n\varepsilon_i^2 \\
        & \text{s.t.} \\
        &  \gamma_i^{'}y_i = \alpha_i + \beta_i^{'}X_i - \varepsilon_i \quad \forall i \\
        &  \alpha_i + \beta_i^{'}X_i -\gamma_i^{'}y_i \le \alpha_j + \beta_j^{'}X_i -\gamma_j^{'}y_i \quad  \forall i, j\\
        &  \gamma_i^{'} g^{y}  + \beta_i^{'} g^{x}  = 1  \quad \forall i \\ 
        &  \beta_i \ge 0 , \gamma_i \ge 0 \quad  \forall i \\
    \end{align*}

Here the residual $\hat{\varepsilon}_i$ represents the estimated value of `d`
($\vec{D}(x_i,y_i,g^x,g^y)+u_i$). We also introduce new firm-specific coefficients
$\gamma_i$ that represent marginal effects of outputs to the DDF. 
The first constraint defines the distance to the frontier as a linear function of inputs 
and outputs. The linear approximation of the frontier is based on the tangent hyperplanes, 
analogous to the original CNLS formulation. The second set of constraints is the 
system of Afriat inequalities thatimpose global concavity. The third constraint 
is a normalization constraint that ensures the translation property. The last 
two constraints impose monotonicity in allinputs and outputs. It is straightforward 
to show that the CNLS estimator of function `d` satisfies the axioms of free disposability, 
convexity, and the translation property.


When considering undesirable outputs, the above CNLS-DDF problem can be reformulated as
    \begin{align*}
        & \underset{\alpha, \beta, \varepsilon} {min} \sum_{i=1}^n\varepsilon_i^2 \\
        & \text{s.t.} \\
        &  \gamma_i^{'}y_i = \alpha_i + \beta_i^{'}X_i + \delta_i^{'}b_i - \varepsilon_i \quad \forall i \\
        &  \alpha_i + \beta_i^{'}X_i + \delta_i^{'}b_i -\gamma_i^{'}y_i \le \alpha_j + \beta_j^{'}X_i + \delta_j^{'}b_i -\gamma_j^{'}y_i \quad  \forall i, j\\
        &  \gamma_i^{'} g^{y}  + \beta_i^{'} g^{x} + \delta_i^{'}g^{b} = 1  \quad \forall i \\ 
        &  \beta_i \ge 0, \delta_i \ge 0, \gamma_i \ge 0 \quad  \forall i \\
    \end{align*}

In [2]:
# import the package pystoned
from pystoned import CNLSDDF
import pandas as pd
import numpy as np

In [3]:
# import Finnish electricity distribution firms data
url='https://raw.githubusercontent.com/ds2010/pyStoNED/master/sources/data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03


In [4]:
# outputs
y1 = df['Energy']
y1 = np.asmatrix(y1).T
y2 = df['Length']
y2 = np.asmatrix(y2).T
y3 = df['Customers']
y3 = np.asmatrix(y3).T
y = np.concatenate((y1, y2, y3), axis=1)

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [6]:
# define and solve the CNLS-DDF model
model = CNLSDDF.CNLSDDF(y, x, b=None, fun = "prod", gx= [0.0, 0.0], gy= [0.0, 0.5, 0.5], gb=None)
model.optimize(remote=False)

Estimating the model locally with mosek solver
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 8010            
  Cones                  : 0               
  Scalar variables       : 623             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 89
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. d

In [7]:
# display the estimates (alpha, beta, gamma, and residual)
model.display_alpha()

alpha : alpha
    Size=89, Index=I
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :  None :  -536.6169000359416 :  None : False : False :  Reals
      1 :  None :  -599.1195598717961 :  None : False : False :  Reals
      2 :  None :  -552.8344077913242 :  None : False : False :  Reals
      3 :  None :  -489.4713431485214 :  None : False : False :  Reals
      4 :  None : -505.14619186968315 :  None : False : False :  Reals
      5 :  None :   -549.809828385151 :  None : False : False :  Reals
      6 :  None :   -489.581822274489 :  None : False : False :  Reals
      7 :  None :  -526.7414710367968 :  None : False : False :  Reals
      8 :  None :  -489.5090236520573 :  None : False : False :  Reals
      9 :  None :  -540.6118569746179 :  None : False : False :  Reals
     10 :  None :   5013.920051219489 :  None : False : False :  Reals
     11 :  None :  11707.559586622856 :  None : False : False :  Reals
     12 :  None :  -558.4392797998554 :  N

In [8]:
model.display_beta()

beta : beta
    Size=178, Index=beta_index
    Key     : Lower : Value                  : Upper : Fixed : Stale : Domain
     (0, 0) :   0.0 :     0.6951278849678328 :  None : False : False :  Reals
     (0, 1) :   0.0 :      4.018951733879629 :  None : False : False :  Reals
     (1, 0) :   0.0 :     1.5157516429733902 :  None : False : False :  Reals
     (1, 1) :   0.0 :      3.741771626533724 :  None : False : False :  Reals
     (2, 0) :   0.0 :    0.21712078903632726 :  None : False : False :  Reals
     (2, 1) :   0.0 :      4.272103824984447 :  None : False : False :  Reals
     (3, 0) :   0.0 :    0.02285292187492183 :  None : False : False :  Reals
     (3, 1) :   0.0 :     3.7793375568006278 :  None : False : False :  Reals
     (4, 0) :   0.0 :     1.8149014470537446 :  None : False : False :  Reals
     (4, 1) :   0.0 :     2.2117867760661185 :  None : False : False :  Reals
     (5, 0) :   0.0 :     1.6332268348338699 :  None : False : False :  Reals
     (5, 1) :   0.0 :

In [9]:
model.display_gamma()

gamma : gamma
    Size=267, Index=gamma_index
    Key     : Lower : Value                  : Upper : Fixed : Stale : Domain
     (0, 0) :   0.0 :      7.508677035037963 :  None : False : False :  Reals
     (0, 1) :   0.0 :     1.9036118500980874 :  None : False : False :  Reals
     (0, 2) :   0.0 :    0.09638814990191254 :  None : False : False :  Reals
     (1, 0) :   0.0 :  0.0026095591927316284 :  None : False : False :  Reals
     (1, 1) :   0.0 :      1.738090271158845 :  None : False : False :  Reals
     (1, 2) :   0.0 :     0.2619097288411549 :  None : False : False :  Reals
     (2, 0) :   0.0 :     1.2285085281770942 :  None : False : False :  Reals
     (2, 1) :   0.0 :     1.8415269705966926 :  None : False : False :  Reals
     (2, 2) :   0.0 :    0.15847302940330749 :  None : False : False :  Reals
     (3, 0) :   0.0 :      4.278513859423042 :  None : False : False :  Reals
     (3, 1) :   0.0 :      1.999999835025392 :  None : False : False :  Reals
     (3, 2) :   0.

In [11]:
model.display_residual()

epsilon : residuals
    Size=89, Index=I
    Key : Lower : Value                 : Upper : Fixed : Stale : Domain
      0 :  None :      156.576278145299 :  None : False : False :  Reals
      1 :  None :    -519.7658235038116 :  None : False : False :  Reals
      2 :  None :     957.1744962822634 :  None : False : False :  Reals
      3 :  None :     3403.007958458881 :  None : False : False :  Reals
      4 :  None :    113.20672052421554 :  None : False : False :  Reals
      5 :  None :   -366.25005255211636 :  None : False : False :  Reals
      6 :  None :    -342.4833731920117 :  None : False : False :  Reals
      7 :  None :     352.6059443744641 :  None : False : False :  Reals
      8 :  None :    1010.1933226409527 :  None : False : False :  Reals
      9 :  None :     560.9779643574586 :  None : False : False :  Reals
     10 :  None :    1364.2220779156778 :  None : False : False :  Reals
     11 :  None :    -878.7427172533498 :  None : False : False :  Reals
     12 : 